In [1]:
import cv2
import numpy as np 
from matplotlib import pyplot as plt
import math
from math import *
import random

In [2]:
def show(img):
    plt.imshow(img)
    plt.show()
def showg(img):
    plt.imshow(img,cmap='gray')
    plt.show()
def imshow(img):
    cv2.imshow('ImageWindow',img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [3]:
img = cv2.imread("data/check.bmp")
simA = cv2.imread("data/simA.jpg")
simB = cv2.imread("data/simB.jpg")
transA = cv2.imread("data/transA.jpg")
transB = cv2.imread("data/transB.jpg")

In [4]:
bsimA = cv2.GaussianBlur(simA,(3,3),0)
btransA = cv2.GaussianBlur(transA,(3,3),0)

In [5]:
def derivatives(img):
    scale = 1
    delta = 0
    ddepth = cv2.CV_16S
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    grad_x = cv2.Sobel(gray, ddepth, 1, 0, ksize=3, scale=scale, delta=delta, borderType=cv2.BORDER_DEFAULT)
    grad_y = cv2.Sobel(gray, ddepth, 0, 1, ksize=3, scale=scale, delta=delta, borderType=cv2.BORDER_DEFAULT)
    return grad_x,grad_y

In [6]:
def harris(img,x,y):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    Ixx = x * x
    Iyy = y * y
    Ixy = x * y
    Ixx = np.asarray(Ixx)
    Iyy = np.asarray(Iyy)
    Ixy = np.asarray(Ixy)
    w = 3
    off = int(w/2)
    h,w = img.shape
    Sxx = np.zeros(shape=(h,w))
    Syy = np.zeros(shape=(h,w))
    Sxy = np.zeros(shape=(h,w))
    for i in range (off,h-off):
        for j in range(off,w-off):
            Sxx[i,j] = np.sum(Ixx[i-off:i+1+off, j-off:j+1+off])
            Syy[i,j] = np.sum(Iyy[i-off:i+1+off, j-off:j+1+off])
            Sxy[i,j] = np.sum(Ixy[i-off:i+1+off, j-off:j+1+off])
    det = (Sxx*Syy) - (Sxy**2)
    trace = Sxx+Syy
    k = 0.04
    R = det - k*(trace**2)
    return R

In [7]:
def hit(image, temp,a,b):
    c = 0
    h_t, w_t = temp.shape
    for u in range(0,w_t):
        for v in range(0,h_t):
            if image[b-h_t//2+v,a-w_t//2+u]>c:
                c=image[b-h_t//2+v,a-w_t//2+u]
    return c
def dilation(img,t):
    h,w  = img.shape
    h_t, w_t = t.shape
    empty = np.zeros((h,w))
    for i in range(w_t//2,w-w_t//2):
        for j in range(h_t//2,h-h_t//2):
            c = hit(img,t,i,j)
            empty[j,i] = c
    return empty  

In [8]:
def threshNMS(img,R):
    Rf = R.flatten()
    mx = Rf.max()
    mx =  0.2*mx
    h,w = R.shape
    R1 = np.zeros(shape=(h,w))
#     img = cv2.imread("simA.jpg")
    h,w = R.shape
    for i in range(h):
        for j in range(w):
            if(R[i][j]>mx):
                R1[i][j] = R[i][j]
    t = np.ones(shape=(5,5))
    dR1 = dilation(R1,t)
    h,w = R1.shape
    NMS = np.zeros((h,w))
    count = 0
    for i in range(h):
        for j in range(w):
            if dR1[i][j]==R1[i][j] and dR1[i][j]!=0:
                NMS[i][j]=1
#                 img[i][j] = [255,0,0]
                count+=1
#                 print(i,j)
#     showg(img)
#     print(count)
    return NMS,count

In [9]:
def keypoints(NMS,x,y):
    kps = []
    tans = cv2.phase(np.array(x, np.float32), np.array(y, dtype=np.float32), angleInDegrees=True)
    count = 0
    h,w = NMS.shape
    for i in range(h):
        for j in range(w):
            if NMS[i][j]>0:
                count+=1
                point = cv2.KeyPoint(x=j,y=i, _size = 3, _angle = tans[i][j], _octave=0)
                kps.append(point)
#     print(count)
    return kps
                

In [15]:
def sift1(img):
    x,y = derivatives(img)
    R = harris(img,x,y)
    C, num = threshNMS(img, R)
#     print(num)
    points = keypoints(C,x,y)
    des_img = np.zeros((img.shape[0],img.shape[1],3), dtype=np.uint8)
    cv2.drawKeypoints(img,points, des_img)
    imshow(des_img)
    sift = cv2.xfeatures2d.SIFT_create()
    points,descs = sift.compute(img,points)
    return points,descs

In [16]:
def match(img1,img2):
    sift = cv2.xfeatures2d.SIFT_create()
    pointsA, descA = sift1(img1)
    pointsB, descB = sift1(img2)
    bfm = cv2.BFMatcher()
    matches = bfm.match(descA, descB)
    matches = sorted(matches, key = lambda x:x.distance)
    img3 = cv2.drawMatches(img1,pointsA,img2,pointsB,matches[:10], None,flags=2)
    return img3,matches[:10],pointsA,pointsB

In [17]:
a = sift1(transA)

In [296]:
image,matches,kp1,kp2 = match(transA,transB)

In [297]:
imshow(image)

In [299]:
list_kp1 = []
list_kp2 = []

for mat in matches:

    # Get the matching keypoints for each of the images
    img1_idx = mat.queryIdx
    img2_idx = mat.trainIdx

    # x - columns
    # y - rows
    # Get the coordinates
    (x1, y1) = kp1[img1_idx].pt
    (x2, y2) = kp2[img2_idx].pt

    # Append to each list
    list_kp1.append((x1, y1))
    list_kp2.append((x2, y2))

In [302]:
l = len(list_kp1)
l

10

In [303]:
csmx = 0
diff = [0,0]
idx = -1
for i in range(5):
    j = random.randrange(0,10,1)
    d = [0,0]
    cs = 0
    print(list_kp1[j],list_kp2[j])
    d[0] = list_kp2[j][0]-list_kp1[j][0]
    d[1] = list_kp2[j][1]-list_kp1[j][1]
    for k in range(l):
        d1 = list_kp2[k][0]-list_kp1[k][0]
        d2 = list_kp2[k][1]-list_kp1[k][1]
        if abs(d1-d[0])<20 and abs(d2-d[1]<20):
            cs+=1
    if(cs>csmx):
        csmx = cs
        diff = d
        idx = j
csmx

(608.0, 212.0) (443.0, 135.0)
(552.0, 393.0) (393.0, 297.0)
(552.0, 393.0) (393.0, 297.0)
(287.0, 470.0) (154.0, 364.0)
(185.0, 260.0) (37.0, 176.0)


8

In [304]:
T = np.float32([[1, 0, diff[0]], [0, 1, diff[1]]]) 
h,w = transA.shape[:2]
ttransA = cv2.warpAffine(transA, T, (w, h))
imshow(ttransA)
csmx

8

In [307]:
T

array([[   1.,    0., -148.],
       [   0.,    1.,  -84.]], dtype=float32)

In [305]:
image,matches,kp1,kp2 = match(ttransA,transB)

In [306]:
imshow(image)